The structure of this notebook goes as follows:
1) Load in fastq.gz files, assign relevant information, and cut/trim the raw sequences
2) Inspecting the nature and quality of all your samples and decide parameters of further analysis
3) Assign taxa with reference classifier, then calculate core diversity metrics
4) Visualise alpha/beta diversity
5) Calculate permanova/betadisper (which helps explain the result of diversity metrics)
6) Identify differential taxa (specific microbes that differ in subsets of your samples)
7) Create cladogram

Section 1 we must create  a "manifest.tsv" (which provides paths to the actual sequence files) and a "metadata.tsv" (which provides details to the experiment which gives context to each sample in analysis). These two files will guide the entire analysis pipeline

In [2]:
# Import all necessary packages
import os
import re
import pandas as pd
from pathlib import Path
import biom
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from matplotlib.patches import Ellipse
import glob
import csv
import json
import qiime2
import seaborn as sns
from skbio.stats.ordination import OrdinationResults
import shutil

In [29]:
import os
import re
import pandas as pd
from pathlib import Path

# Get notebook directory
project_dir = Path(os.getcwd())

# Define input/output dirs
raw_dir = project_dir / "raw"
qiime_dir = project_dir / 'qiime'
manifest_file = project_dir / "manifest.tsv"

# Recursively list all FASTQ files
fastqs = sorted(raw_dir.rglob("*.fq.gz"))

if not fastqs:
    raise FileNotFoundError(f"No FASTQ files found in {raw_dir}")

# Helper: get parent folder name as sampleid
def extract_sample_name(fname: Path) -> str:
    """
    Use the parent folder name of the FASTQ file as the sample ID.
    """
    return fname.parent.name

# Group by sample name (folder containing the reads)
samples = {}
for f in fastqs:
    sample = extract_sample_name(f)
    if sample not in samples:
        samples[sample] = {"R1": None, "R2": None}

    if re.search(r"(R1|_1)\.f(ast)?q\.gz$", f.name):
        samples[sample]["R1"] = f.resolve()
    elif re.search(r"(R2|_2)\.f(ast)?q\.gz$", f.name):
        samples[sample]["R2"] = f.resolve()

# Build manifest DataFrame
records = []
for sample, files in samples.items():
    if files["R1"] and files["R2"]:
        records.append({
            "sampleid": sample,
            "forward-absolute-filepath": str(files["R1"]),
            "reverse-absolute-filepath": str(files["R2"]),
        })
    else:
        print(f"⚠️ Skipping {sample}: missing R1 or R2")

manifest = pd.DataFrame(records)

# Save manifest
manifest.to_csv(manifest_file, sep="\t", index=False)

print(f"✅ Manifest written to: {manifest_file.resolve()}")
print(f"Samples included: {len(manifest)}")
print(manifest.head())


✅ Manifest written to: /home/patwuch/projects/microbiome/experiments/yang/manifest.tsv
Samples included: 20
   sampleid                          forward-absolute-filepath  \
0  PD_O_BK2  /home/patwuch/projects/microbiome/experiments/...   
1  PD_O_BK4  /home/patwuch/projects/microbiome/experiments/...   
2  PD_O_BU2  /home/patwuch/projects/microbiome/experiments/...   
3   PD_O_G1  /home/patwuch/projects/microbiome/experiments/...   
4  PD_Sh_G1  /home/patwuch/projects/microbiome/experiments/...   

                           reverse-absolute-filepath  
0  /home/patwuch/projects/microbiome/experiments/...  
1  /home/patwuch/projects/microbiome/experiments/...  
2  /home/patwuch/projects/microbiome/experiments/...  
3  /home/patwuch/projects/microbiome/experiments/...  
4  /home/patwuch/projects/microbiome/experiments/...  


Section 2 we start actually working with our sequences. We will inspect quality, denoise, truncate, and inspect again.

In [5]:
# This imports the fastq files and creates a QIIME2 artifact
!mkdir -p qiime
# Select the manifest 
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-format PairedEndFastqManifestPhred33V2 \
  --input-path "manifest.tsv" \
  --output-path "qiime/demux.qza"

/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Imported manifest.tsv as PairedEndFastqManifestPhred33V2 to qiime/demux.qza


In [6]:
# This creates a qiime2 visualisation artefact with which you can check on QIIME2 View the quality of the reads
# Based on the QC plots, you can decide the length at which to trim the reads in the next step
!qiime demux summarize \
    --i-data "qiime/demux.qza" \
    --o-visualization "qiime/demux.qzv"

/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: qiime/demux.qzv


In [7]:
# DADA2 is a popular denoising algorithm that corrects amplicon errors while also removing chimeras
# It will return artefacts which can be converted to visualisation to see if we trimmed appropriately
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs "qiime/demux.qza" \
    --p-trim-left-f 10 \
    --p-trim-left-r 10 \
    --p-trunc-len-f 260 \
    --p-trunc-len-r 220 \
    --o-table "qiime/table.qza" \
    --o-representative-sequences "qiime/rep-seqs.qza" \
    --o-denoising-stats "qiime/stats.qza" \
    --p-n-threads 4

/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: qiime/table.qza
Saved FeatureData[Sequence] to: qiime/rep-seqs.qza
Saved SampleData[DADA2Stats] to: qiime/stats.qza


In [3]:
# Now we convert the dada2 artefacts in the previous cell to visualisations
# If it passes the QC, we can proceed to assign taxonomy
!qiime feature-table summarize \
    --i-table "qiime/table.qza" \
    --o-visualization "qiime/table.qzv" \
    --m-sample-metadata-file "metadata.tsv"

!qiime feature-table tabulate-seqs \
    --i-data "qiime/rep-seqs.qza" \
    --o-visualization "qiime/rep-seqs.qzv"

!qiime metadata tabulate \
    --m-input-file "qiime/stats.qza" \
    --o-visualization "qiime/stats.qzv"

/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: qiime/table.qzv
/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: qiime/rep-seqs.qzv
/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_re

In [1]:
# Point this to your actual classifier
classifier_path = "/home/patwuch/projects/microbiome/reference/gg_12_10_primer_region-classifier.qza"

!qiime feature-classifier classify-sklearn \
    --i-classifier {classifier_path} \
    --i-reads "qiime/rep-seqs.qza" \
    --o-classification "qiime/taxonomy.qza"

# Exhaustive list of taxonomic assignments with confidence scores
!qiime metadata tabulate \
    --m-input-file "qiime/taxonomy.qza" \
    --o-visualization "qiime/taxonomy.qzv"


QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Taxonomy] to: qiime/taxonomy.qza
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: qiime/taxonomy.qzv


In [3]:
# Taxa bar plots are a common way to visualise taxonomic composition across samples
!qiime taxa barplot \
    --i-table "qiime/table.qza" \
    --i-taxonomy "qiime/taxonomy.qza" \
    --m-metadata-file "metadata.tsv" \
    --o-visualization "qiime/taxa-bar-plots.qzv"

# Krona is a helpful way to interactively explore taxonomic composition
# But it is not great for static figures in publications
!qiime krona collapse-and-plot \
    --i-table "qiime/table.qza" \
    --i-taxonomy "qiime/taxonomy.qza" \
    --o-krona-plot "qiime/krona.qzv"


/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: qiime/taxa-bar-plots.qzv
Error: QIIME 2 has no plugin/command named 'krona'.


In [4]:
# If you want static plots of alpha and beta diversity as png, you must use the following commands
# These export qiime artefacts into more standard bioinformatics file formats
!qiime tools export \
  --input-path qiime/table.qza \
  --output-path exported/exported-feature-table

!qiime tools export \
  --input-path qiime/taxonomy.qza \
  --output-path exported/exported-taxonomy

/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported qiime/table.qza as BIOMV210DirFmt to directory exported/exported-feature-table
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported qiime/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory exported/exported-taxonomy


In [26]:
# Then we convert .biom to .tsv (tsv is the most generalist format--which means it'll be easy to pass to R)
!biom convert \
  -i exported/exported-feature-table/feature-table.biom \
  -o exported/exported-feature-table/table.from_biom.tsv \
  --to-tsv
# Same goes for taxonomy, 

In [5]:
import os
import pandas as pd
import biom
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

output_dir = "taxa_barplots"
os.makedirs(output_dir, exist_ok=True)

# ===========================
# USER SETTINGS
# ===========================
feature_table_biom = "exported/exported-feature-table/feature-table.biom"
taxonomy_tsv = "exported/exported-taxonomy/taxonomy.tsv"
top_n = 20            # Number of taxa to show in legend

# ===========================
# LOAD DATA
# ===========================
table = biom.load_table(feature_table_biom)
df = pd.DataFrame(table.matrix_data.toarray().T, 
                  index=table.ids(axis='sample'), 
                  columns=table.ids(axis='observation'))

taxonomy = pd.read_csv(taxonomy_tsv, sep='\t', index_col=0)

# 👇 FIX FOR AttributeError: 'float' object has no attribute 'lower'
# Ensure the Taxon column is treated as a string and missing values are empty strings.
taxonomy['Taxon'] = taxonomy['Taxon'].astype(str).fillna('').str.strip()

level_dict = {
    "Kingdom": 0,
    "Phylum": 1,
    "Class": 2,
    "Order": 3,
    "Family": 4,
    "Genus": 5,
}

# Define the order of levels for easy lookup
LEVEL_NAMES = list(level_dict.keys())
LEVEL_INDICES = list(level_dict.values())

# Helper function to find the highest-level assignment for a feature
def relabel_unassigned_taxa(row, current_level_index):
    """
    Looks up the next available, higher taxonomic level for unassigned features,
    safely handling taxonomy strings that are shorter than 6 levels.
    """
    
    # Safely split the taxonomy string and pad with 'Unassigned' if needed.
    taxa_list = [t.strip() for t in row['Taxon'].split(';')]
    # Ensure the list is padded up to the maximum number of levels (6)
    while len(taxa_list) < len(LEVEL_NAMES): 
        taxa_list.append("Unassigned")
        
    # 1. Get the current level assignment
    current_label = taxa_list[current_level_index]
    
    # 2. Check if the current label is unassigned (or similar)
    if 'unassigned' in current_label.lower() or current_label.strip() == '':
        # 3. Iterate through higher levels (from current_level_index - 1 down to 0)
        for i in range(current_level_index - 1, -1, -1):
            higher_level_label = taxa_list[i]
            
            # 4. If a higher level is assigned, use it as the prefix
            if 'unassigned' not in higher_level_label.lower() and higher_level_label.strip() != '':
                higher_level_name = LEVEL_NAMES[i]
                # Format: "Unassigned ({HigherLevel} {HigherLabel})"
                return f"Unassigned ({higher_level_name} {higher_level_label})"
        
        # 5. If no higher level is assigned (all are unassigned), return a generic label
        return "Unassigned (All)"
    
    # 6. If the current label is not unassigned, return it as is
    return current_label


# ===========================
# LOOP THROUGH TAXONOMIC LEVELS
# ===========================
for tax_level, level_index in level_dict.items():
    print(f"Processing {tax_level}...")

    # --- LOGIC FOR RELABELING ---
    if tax_level != "Kingdom": # Kingdom level is the highest, no higher level to check
        # Apply the relabeling function to create the new, more descriptive tax column
        taxonomy[tax_level] = taxonomy.apply(
            relabel_unassigned_taxa, 
            axis=1, 
            current_level_index=level_index
        )
    else:
        # Kingdom level is simpler: just get the label and handle Unassigned
        # We rely on the padding in the function below to ensure safe access,
        # but for Kingdom we can still use the simpler logic for readability.
        taxonomy[tax_level] = taxonomy['Taxon'].str.split(';').str[level_index].str.strip().fillna("Unassigned")
        taxonomy[tax_level] = taxonomy[tax_level].apply(
            lambda x: "Unassigned (All)" if 'unassigned' in x.lower() or x.strip() == '' else x
        )
    
    # --- Special case for Genus (Family|Genus) still needed ---
    if tax_level == "Genus":
        # Extract the Family name. We use the safe list access within a lambda.
        taxonomy["Family_prefix"] = taxonomy['Taxon'].apply(
            lambda x: ([t.strip() for t in x.split(';')] + ['Unassigned'] * 6)[level_dict["Family"]]
        )
        
        # We need the *relabelled* genus for the second part of the string
        # Combine Family prefix with the (potentially relabelled) Genus name
        taxonomy[tax_level] = taxonomy.apply(
            lambda row: f"{row['Family_prefix']}|{row[tax_level]}" 
                        if 'unassigned' not in row['Family_prefix'].lower() and row['Family_prefix'].strip() != ''
                        else row[tax_level], # If Family is unassigned, just use the Genus label
            axis=1
        )
    # -----------------------------------------------------------

    # Aggregate counts by taxonomic level
    df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
    df_tax_norm = df_tax.div(df_tax.sum(axis=1), axis=0)

    # Identify top N taxa by mean relative abundance
    mean_abundance = df_tax_norm.mean(axis=0)
    top_taxa = mean_abundance.sort_values(ascending=False).head(top_n).index

    # Prepare dataframe for plotting
    df_top = df_tax_norm[top_taxa].copy()
    df_top['Other'] = df_tax_norm.drop(columns=top_taxa, errors='ignore').sum(axis=1)

    # Color map
    n_colors = df_top.shape[1]
    cmap = cm.get_cmap('tab20', n_colors)
    colors = [cmap(i) for i in range(n_colors)]

    # Plot manually stacked bars
    fig, ax = plt.subplots(figsize=(12, 6))
    bottom = np.zeros(df_top.shape[0])
    for i, col in enumerate(df_top.columns):
        ax.bar(
            df_top.index,
            df_top[col],
            bottom=bottom,
            color=colors[i],
            label=col,
            width=0.8,
            edgecolor="none",
            linewidth=0,
            antialiased=False
        )
        bottom += df_top[col].values

    # Axis labels and title
    ax.set_ylabel("Relative abundance")
    ax.set_xlabel("Samples")
    plt.xticks(rotation=90)
    plt.title(f"Relative abundance at {tax_level} level")

    # Legend in descending abundance order
    handles, labels = ax.get_legend_handles_labels()
    handles, labels = handles[::-1], labels[::-1]
    ax.legend(
        handles, labels,
        bbox_to_anchor=(1.05, 1),
        loc='upper left',
        fontsize=8,
        title="Taxa",
        title_fontsize=9,
        frameon=False
    )

    plt.tight_layout()
    output_png = f"{output_dir}/taxa_barplot_{tax_level}.png"
    plt.savefig(output_png, dpi=300, bbox_inches="tight")
    plt.close()

Processing Kingdom...


/tmp/ipykernel_2564027/1625060123.py:121: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
/tmp/ipykernel_2564027/1625060123.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', n_colors)


Processing Phylum...


/tmp/ipykernel_2564027/1625060123.py:121: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
/tmp/ipykernel_2564027/1625060123.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', n_colors)


Processing Class...


...

/tmp/ipykernel_2564027/1625060123.py:121: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
/tmp/ipykernel_2564027/1625060123.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', n_colors)


Processing Family...


/tmp/ipykernel_2564027/1625060123.py:121: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
/tmp/ipykernel_2564027/1625060123.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', n_colors)


Processing Genus...


/tmp/ipykernel_2564027/1625060123.py:121: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_tax = df.groupby(taxonomy[tax_level], axis=1).sum()
/tmp/ipykernel_2564027/1625060123.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', n_colors)


In [ ]:
# builds greengenes2 phylogenetic tree
# THIS MUST BE RUN IN THE Qiime2 Greengenes2 ENVIRONMENT or else it will break core dependencies for other parts of the pipeline
# Greengenes2 provides a either v4 or non-v4 backbone for phylogenetic placement, which can essentially be used for any 16S region
# Unless you have too much computation power  go train a new tree then lol

!qiime greengenes2 non-v4-16s \
    --i-table qiime/table.qza \
    --i-sequences qiime/rep-seqs.qza \
    --i-backbone /home/patwuch/projects/microbiome/reference/Greengenes2/2024.09.backbone.full-length.fna.qza \
    --p-threads 6 \
    --o-mapped-table qiime/gg-table.qza \
    --o-representatives qiime/gg-rep-seqs.qza

/home/patwuch/miniforge3/envs/q2-amplicon-gg-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: qiime/gg-table.qza
Saved FeatureData[Sequence] to: qiime/gg-rep-seqs.qza


In [1]:
!rm -rf core-metrics-results
!mkdir core-metrics-results


# First we must rarefy the feature table to an even sampling depth
!qiime feature-table rarefy \
  --i-table qiime/gg-table.qza \
  --p-sampling-depth 44645 \
  --o-rarefied-table qiime/rarefied_table.qza

# Shannon
!qiime diversity alpha \
  --i-table qiime/rarefied_table.qza \
  --p-metric shannon \
  --o-alpha-diversity core-metrics-results/shannon_vector.qza

# Chao1
!qiime diversity alpha \
  --i-table qiime/rarefied_table.qza \
  --p-metric chao1 \
  --o-alpha-diversity core-metrics-results/chao1_vector.qza

# Simpson
!qiime diversity alpha \
  --i-table qiime/rarefied_table.qza \
  --p-metric simpson \
  --o-alpha-diversity core-metrics-results/simpson_vector.qza

# Evenness
!qiime diversity alpha \
  --i-table qiime/rarefied_table.qza \
  --p-metric pielou_e \
  --o-alpha-diversity core-metrics-results/evenness_vector.qza

/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: qiime/rarefied_table.qza
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: 

In [ ]:
phylogeny_path = "/home/patwuch/projects/microbiome/reference/Greengenes2/2024.09.phylogeny.id.nwk.qza"

!qiime diversity check-tree \
  --i-table qiime/rarefied-table.qza \
  --i-phylogeny "$phylogeny_path"
# Now for beta diversity
# Jaccard
# !qiime diversity beta \
#   --i-table qiime/rarefied_table.qza \
#   --p-metric jaccard \
#   --o-distance-matrix core-metrics-results/jaccard_distance_matrix.qza


# # Bray-Curtis
# !qiime diversity beta \
#   --i-table qiime/rarefied_table.qza \
#   --p-metric braycurtis \
#   --o-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza

# !qiime diversity pcoa \
#   --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
#   --o-pcoa core-metrics-results/bray_curtis_pcoa_results.qza
# !qiime diversity pcoa \
#   --i-distance-matrix core-metrics-results/jaccard_distance_matrix.qza \
#   --o-pcoa core-metrics-results/jaccard_pcoa_results.qza


#   BETA DIVERSITY (Phylogenetic - using the full Greengenes2 tree because we used SEPP on the non-v4 backbone too)
!qiime diversity beta-phylogenetic \
  --i-table qiime/rarefied_table.qza \
  --i-phylogeny /home/patwuch/projects/microbiome/reference/Greengenes2/2024.09.phylogeny.id.nwk.qza\
  --p-metric unweighted_unifrac \
  --o-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --p-threads 1

# BETA DIVERSITY (Phylogenetic - using the Greengenes2 tree)
!qiime diversity beta-phylogenetic \
  --i-table qiime/rarefied_table.qza \
  --i-phylogeny "$phylogeny_path" \
  --p-metric weighted_unifrac \
  --o-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --p-threads 1




!qiime diversity pcoa \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --o-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza
!qiime diversity pcoa \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --o-pcoa core-metrics-results/weighted_unifrac_pcoa_results.qza



Error: QIIME 2 plugin 'diversity' has no action 'check-tree'.
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Plugin error from diversity:

  Command '['ssu', '-i', '/tmp/qiime2/root/data/7ad98397-dfa6-49c3-9497-6126da3a205b/data/feature-table.biom', '-t', '/tmp/qiime2/root/data/52e0fb90-d576-4f77-a07e-8f0b9dd78dda/data/tree.nwk', '-m', 'unweighted', '-o', '/tmp/qiime2/root/processes/2685807-1762243564.03@root/tmp/q2-OutPath-owvqkimz']' died with <Signals.SIGABRT: 6>.

Debug info has been saved to /tmp/qiime2-q2cli-err-0hx7iisn.log
/home/patwuch/miniforge3/envs/q2-amplicon-core-2025.10/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is d

Section 4 we create alpha and beta diversity visualisations.

In [3]:
# ---- Load alpha diversity artifacts ----
shannon = qiime2.Artifact.load("core-metrics-results/shannon_vector.qza").view(pd.Series)
evenness = qiime2.Artifact.load("core-metrics-results/evenness_vector.qza").view(pd.Series)
chao1 = qiime2.Artifact.load("core-metrics-results/chao1_vector.qza").view(pd.Series)
simpson = qiime2.Artifact.load("core-metrics-results/simpson_vector.qza").view(pd.Series)

# ---- Load metadata ----
metadata = pd.read_csv("metadata.tsv", sep="\t", index_col=0)

# ---- Combine alpha diversity into one DataFrame ----
alpha_df = pd.concat([
    shannon.rename("Shannon"),
    evenness.rename("Evenness"),
    chao1.rename("Chao1"),
    simpson.rename("Simpson")
], axis=1)

# ---- Merge alpha diversity with metadata ----
merged = alpha_df.join(metadata)
merged['Group'] = merged['Group'].str.strip()

# ---- Output folder ----
os.makedirs("alpha_diversity_plots", exist_ok=True)
sns.set(style="whitegrid")

# ---- Define comparisons in flexible style ----
comparisons = [
    ("Group", None),                # all groups
    ("Butyrate", None),  # only C subgroups
    ("Disease", None)      # only E subgroups
]

# ---- Function to plot alpha diversity ----
def plot_alpha_diversity(df, x_col, levels=None, title=None, outfile=None):
    # Subset if levels provided
    if levels is not None:
        df = df[df[x_col].isin(levels)]
    
    # Melt for seaborn
    melted = df.melt(
        id_vars=[x_col],
        value_vars=["Shannon", "Evenness", "Chao1", "Simpson"],
        var_name="Metric",
        value_name="Diversity"
    )
    
    # Plot
    g = sns.catplot(
        data=melted,
        x=x_col, y="Diversity",
        col="Metric",
        kind="box",
        col_wrap=2,
        sharey=False,
        height=4, aspect=1.2
    )
    g.map_dataframe(sns.stripplot, x=x_col, y="Diversity", color="black", alpha=0.5)
    plt.subplots_adjust(top=0.85)
    
    if title:
        g.figure.suptitle(title)
    if outfile:
        g.savefig(outfile, dpi=300, bbox_inches="tight")
    plt.close(g.fig)

# ---- Loop over comparisons ----
for x_col, levels in comparisons:
    name = f"{x_col}_{'_'.join(levels) if levels else 'all'}"
    title = f"Alpha Diversity - {name.replace('_', ' ')}"
    outfile = f"alpha_diversity_plots/alpha_diversity_{name}.png"
    
    plot_alpha_diversity(
        df=merged,
        x_col=x_col,
        levels=levels,
        title=title,
        outfile=outfile
    )


NameError: name 'qiime2' is not defined

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from qiime2 import Artifact
from skbio.stats.ordination import OrdinationResults

# Load PCoA results
pcoa_results = {
    "Bray-Curtis": Artifact.load("core-metrics-results/bray_curtis_pcoa_results.qza").view(OrdinationResults),
    "Jaccard": Artifact.load("core-metrics-results/jaccard_pcoa_results.qza").view(OrdinationResults),
    "Weighted Unifrac": Artifact.load("core-metrics-results/weighted_unifrac_pcoa_results.qza").view(OrdinationResults),
    "Unweighted Unifrac": Artifact.load("core-metrics-results/unweighted_unifrac_pcoa_results.qza").view(OrdinationResults),
}

metadata = pd.read_csv("metadata.tsv", sep="\t", index_col=0)

# Strip whitespace from all string/object columns
for col_name in metadata.select_dtypes(include=['object']).columns:
    metadata[col_name] = metadata[col_name].str.strip()
n_pcs = 19

comparisons = [
    ("Group", None),
    ("Butyrate", None),
    ("Disease", None)
]

output_dir = "beta_diversity_plots"
os.makedirs(output_dir, exist_ok=True)

for col, filter_values in comparisons:
    if col not in metadata.columns:
        print(f"Warning: Column '{col}' not found in metadata. Skipping.")
        continue

    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.flatten()

    for ax, (distance_metric, pcoa_res) in zip(axes, pcoa_results.items()):
        coords = pcoa_res.samples
        df = coords.merge(metadata, left_index=True, right_index=True)
        df.rename(columns={i: f'PC{i+1}' for i in range(n_pcs)}, inplace=True)

        # Filter if needed
        df_subset = df.copy()
        if filter_values is not None:
            df_subset = df_subset[df_subset[col].isin(filter_values)]

        if df_subset.empty:
            print(f"Warning: No data for {col} in {distance_metric}. Skipping plot.")
            ax.set_title(f"{distance_metric} (no data)")
            ax.axis('off')
            continue

        # Ensure PC1 and PC2 exist
        if "PC1" not in df_subset.columns or "PC2" not in df_subset.columns:
            print(f"Warning: PC1 or PC2 missing for {distance_metric}. Skipping.")
            ax.set_title(f"{distance_metric} (no PC1/PC2)")
            ax.axis('off')
            continue

        unique_groups = df_subset[col].dropna().unique()
        palette = sns.color_palette(n_colors=len(unique_groups))
        group_to_color = dict(zip(unique_groups, palette))

        # Scatter plot
        sns.scatterplot(
            x="PC1",
            y="PC2",
            hue=col if not df_subset[col].isnull().all() else None,
            data=df_subset,
            s=100,
            alpha=0.8,
            palette=group_to_color if not df_subset[col].isnull().all() else None,
            ax=ax
        )

        # Draw ellipses
        for group, data_subset in df_subset.groupby(col):
            if len(data_subset) < 2:
                continue
            centroid_x = data_subset["PC1"].mean()
            centroid_y = data_subset["PC2"].mean()
            width = data_subset["PC1"].std() * 2
            height = data_subset["PC2"].std() * 2
            ellipse = Ellipse(
                (centroid_x, centroid_y),
                width=width, height=height,
                edgecolor=group_to_color[group],
                facecolor='none', lw=2, alpha=0.7
            )
            ax.add_patch(ellipse)
            ax.scatter(centroid_x, centroid_y, marker='x', color='black', s=120, zorder=10)

        ax.set_title(distance_metric)
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")

    # Combine legend outside grid
    handles, labels = axes[0].get_legend_handles_labels()
    if handles:
        fig.legend(handles, labels, title=col, bbox_to_anchor=(1.05, 0.5), loc='center left')

    subset_label = "all" if filter_values is None else "_".join(filter_values)
    fig.suptitle(f"PCoA comparison ({col} = {subset_label})", fontsize=16)

    plt.tight_layout(rect=[0, 0, 0.85, 0.95])

    filename = f"PCoA_comparison_{col}_{subset_label}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300)
    plt.close()


In [32]:
# Export rooted tree and taxonomy for use in R or other software
!qiime tools export \
    --input-path "qiime/rooted-tree.qza" \
    --output-path "exported/rooted-tree"
!qiime tools export \
    --input-path "qiime/taxonomy.qza" \
    --output-path "exported/taxonomy"


/home/patwuch/miniforge3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported qiime/rooted-tree.qza as NewickDirectoryFormat to directory exported/rooted-tree
/home/patwuch/miniforge3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported qiime/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory exported/taxonomy


Section 5 we create the PERMANOVA and BETADISPER tables using a combination of python and R, as R creates more legible results than qiime in this case.

!!! The R cell will not run if you do not run the below cell to initiate R magic first !!!

In [14]:
%load_ext rpy2.ipython

In [15]:
core_metrics_dir = "core-metrics-results"
qza_files = [f for f in os.listdir(core_metrics_dir) if f.endswith("_distance_matrix.qza")]

distance_names = []
distance_paths = []

for qza in qza_files:
    name = qza.replace("_distance_matrix.qza", "")
    out_path = os.path.join("exported", f"{name}_distance_matrix.tsv")

    # Make sure export dir exists
    os.makedirs("exported", exist_ok=True)

    # Use a temp dir for the raw export
    tmp_dir = f"_tmp_export_{name}"
    os.makedirs(tmp_dir, exist_ok=True)

    # Export with QIIME2
    !qiime tools export --input-path "{os.path.join(core_metrics_dir, qza)}" --output-path "{tmp_dir}"

    # Move the exported file
    raw_exported = os.path.join(tmp_dir, "distance-matrix.tsv")
    if os.path.exists(raw_exported):
        shutil.move(raw_exported, out_path)

    # Clean up temp dir
    shutil.rmtree(tmp_dir)

    distance_names.append(name)
    distance_paths.append(out_path)


/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported core-metrics-results/aitchison_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory _tmp_export_aitchison
/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported core-metrics-results/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory _tmp_export_jaccard
/home/patwuch/miniforge3/e

In [19]:
%%R -i distance_names -i distance_paths 
library(vegan)
library(dplyr)
library(tibble)

# ------------------------------
# 1. Define your distance matrices
# ------------------------------
# Example:
# distance_names = c("bray", "jaccard")
# distance_paths = c("bray_dist.tsv", "jaccard_dist.tsv")
distance_files <- setNames(as.list(distance_paths), distance_names)

# ------------------------------
# 2. Define comparisons and allowed levels
# ------------------------------
comparisons <- list(
  Group = "Group",
  Butyrate = "Butyrate",
  Disease = "Disease"
)

# Optional: restrict to allowed levels
allowed_levels <- list(
  Group = c("PD_O", "Sh_Or", "Sh_int", "PD_int", "PD_Sh"),
  Butyrate = c("OR","INT","NO"),
  Disease = c("Y","N")
)

# ------------------------------
# 3. Load metadata
# ------------------------------
meta <- read.table("metadata.tsv", header = TRUE, sep = "\t", row.names = 1, check.names = FALSE)

# Clean up potential whitespace
rownames(meta) <- trimws(rownames(meta))

# ------------------------------
# 4. Helper function to load distance matrices
# ------------------------------
load_distance <- function(file) {
  mat <- as.matrix(read.table(file, header = TRUE, row.names = 1, sep = "\t", check.names = FALSE))
  rownames(mat) <- trimws(rownames(mat))
  colnames(mat) <- trimws(colnames(mat))
  as.dist(mat)
}

# ------------------------------
# 5. Run global PERMANOVA + betadisper safely
# ------------------------------
results <- list()

for (dname in names(distance_files)) {
  d <- load_distance(distance_files[[dname]])
  
  for (comp_name in names(comparisons)) {
    col <- comparisons[[comp_name]]
    
    # Subset metadata to allowed levels
    if (!is.null(allowed_levels[[comp_name]])) {
      meta_sub <- meta[meta[[col]] %in% allowed_levels[[comp_name]], , drop = FALSE]
    } else {
      meta_sub <- meta
    }
    
    # Drop NA values
    meta_sub <- meta_sub[!is.na(meta_sub[[col]]), , drop = FALSE]
    
    # Match samples between metadata and distance
    common_samples <- intersect(rownames(meta_sub), labels(d))
    if (length(common_samples) < 2) next
    
    meta_sub <- meta_sub[common_samples, , drop = FALSE]
    d_sub <- as.dist(as.matrix(d)[common_samples, common_samples])
    
    # Skip if fewer than 2 levels remain
    if (length(unique(meta_sub[[col]])) < 2) next
    
    # ---- Global PERMANOVA ----
    fmla <- as.formula(paste("d_sub ~", col))
    ad_global <- tryCatch(
      adonis2(fmla, data = meta_sub, permutations = 9999),
      error = function(e) NULL
    )
    
    if (!is.null(ad_global)) {
      ad_row <- as.data.frame(ad_global[1, ])
      ad_row <- rownames_to_column(ad_row, "Term")
      ad_row$distance <- dname
      ad_row$comparison <- comp_name
      ad_row$scope <- "global"
      ad_row$pair <- NA
      ad_row$test <- "permanova"
      results[[length(results) + 1]] <- ad_row
    }
    
    # ---- Global betadisper ----
    bd_global <- tryCatch(betadisper(d_sub, meta_sub[[col]]), error = function(e) NULL)
    
    if (!is.null(bd_global)) {
      bd_global_anova <- anova(bd_global)
      bd_row <- as.data.frame(bd_global_anova[1, ])
      bd_row <- rownames_to_column(bd_row, "Term")
      bd_row$distance <- dname
      bd_row$comparison <- comp_name
      bd_row$scope <- "global"
      bd_row$pair <- NA
      bd_row$test <- "betadisper"
      results[[length(results) + 1]] <- bd_row
    }
  }
}

# ------------------------------
# 6. Combine and save results
# ------------------------------
if (length(results) > 0) {
  permanova_permdisp_results <- bind_rows(results) %>%
    select(distance, comparison, scope, pair, test, everything())
  
  write.table(
    permanova_permdisp_results,
    file = "permanova_permdisp_global_results.tsv",
    sep = "\t",
    quote = FALSE,
    row.names = FALSE
  )
  
  cat("✅ PERMANOVA and betadisper results written to permanova_permdisp_global_results.tsv\n")
} else {
  cat("⚠️ No valid comparisons were found (possibly due to filtering or missing levels).\n")
}


✅ PERMANOVA and betadisper results written to permanova_permdisp_global_results.tsv


Section 6 to evaluate differential taxa, we use either ANCOMBC or ANCOMBC2.
If you want to get full results and static plots, follow the ANCOMBC2 route.
If you want to get easy visualisation for Qiime2 View, use the ANCOMBC approach.
Either way they should yield similar results so long as your parameters remain consistent.

In [ ]:
# ANCOMBC approach
# -------------------------------
# 1. Compare all 6 groups (Group)
# -------------------------------
!qiime composition ancombc \
  --i-table "qiime/table.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-formula Group \
  --o-differentials "qiime/ancombc-Group-results.qza"

!qiime composition da-barplot \
  --i-data "qiime/ancombc-Group-results.qza" \
  --p-significance-threshold 0.001 \
  --o-visualization "qiime/ancombc-Group-results.qzv"


# -------------------------------
# 2. Compare all 6 groups focusing on MainType (E vs C)
# -------------------------------
!qiime composition ancombc \
  --i-table "qiime/table.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-formula MainType \
  --o-differentials "qiime/ancombc-E-vs-C-results.qza"

!qiime composition da-barplot \
  --i-data "qiime/ancombc-E-vs-C-results.qza" \
  --p-significance-threshold 0.001 \
  --o-visualization "qiime/ancombc-E-vs-C-results.qzv"


# -------------------------------
# 3. Compare Modifier within MainType E
# -------------------------------
!qiime feature-table filter-samples \
  --i-table "qiime/table.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-where "MainType='E'" \
  --o-filtered-table "qiime/table-E.qza"

!qiime composition ancombc \
  --i-table "qiime/table-E.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-formula Modifier \
  --o-differentials "qiime/ancombc-E-Modifier-results.qza"

!qiime composition da-barplot \
  --i-data "qiime/ancombc-E-Modifier-results.qza" \
  --p-significance-threshold 0.001 \
  --o-visualization "qiime/ancombc-E-Modifier-results.qzv"


# -------------------------------
# 4. Compare Modifier within MainType C
# -------------------------------
!qiime feature-table filter-samples \
  --i-table "qiime/table.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-where "MainType='C'" \
  --o-filtered-table "qiime/table-C.qza"

!qiime composition ancombc \
  --i-table "qiime/table-C.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-formula Modifier \
  --o-differentials "qiime/ancombc-C-Modifier-results.qza"

!qiime composition da-barplot \
  --i-data "qiime/ancombc-C-Modifier-results.qza" \
  --p-significance-threshold 0.001 \
  --o-visualization "qiime/ancombc-C-Modifier-results.qzv"


In [1]:
# -------------------------------
# ANCOMBC2 APPROACH
# -------------------------------
# 1. Compare all 6 groups (Group)
# Note ancombc2 automatically uses a ONE vs. REST approach for differential analysis
# -------------------------------
!qiime composition ancombc2 \
  --i-table "qiime/table.qza" \
  --m-metadata-file "metadata.tsv" \
  --p-fixed-effects-formula Group \
  --o-ancombc2-output "qiime/ancombc2-Group-results.qza"


# # -------------------------------
# # 2. Compare all 6 groups focusing on MainType (E vs C)
# # -------------------------------
# !qiime composition ancombc2 \
#   --i-table "qiime/table.qza" \
#   --m-metadata-file "metadata.tsv" \
#   --p-fixed-effects-formula MainType \
#   --o-ancombc2-output "qiime/ancombc2-E-vs-C-results.qza"


# # -------------------------------
# # 3. Compare Modifier within MainType E
# # -------------------------------
# !qiime feature-table filter-samples \
#   --i-table "qiime/table.qza" \
#   --m-metadata-file "metadata.tsv" \
#   --p-where "MainType='E'" \
#   --o-filtered-table "qiime/table-E.qza"

# !qiime composition ancombc2 \
#   --i-table "qiime/table-E.qza" \
#   --m-metadata-file "metadata.tsv" \
#   --p-fixed-effects-formula Modifier \
#   --o-ancombc2-output "qiime/ancombc2-E-Modifier-results.qza"


# # -------------------------------
# # 4. Compare Modifier within MainType C
# # -------------------------------
# !qiime feature-table filter-samples \
#   --i-table "qiime/table.qza" \
#   --m-metadata-file "metadata.tsv" \
#   --p-where "MainType='C'" \
#   --o-filtered-table "qiime/table-C.qza"

# !qiime composition ancombc2 \
#   --i-table "qiime/table-C.qza" \
#   --m-metadata-file "metadata.tsv" \
#   --p-fixed-effects-formula Modifier \
#   --o-ancombc2-output "qiime/ancombc2-C-Modifier-results.qza"

/home/patwuch/miniforge3/envs/qiime2-2025.7/lib/python3.10/site-packages/emperor/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[ANCOMBC2Output] to: qiime/ancombc2-Group-results.qza


In [ ]:
# To create static plots we have to export the ancombc2 artefacts to jsonl files
!qiime tools export \
  --input-path qiime/ancombc2-Group-results.qza \
  --output-path exported/ancombc2-Group-results

!qiime tools export \
  --input-path qiime/ancombc2-E-vs-C-results.qza \
  --output-path exported/ancombc2-E-vs-C-results

!qiime tools export \
  --input-path qiime/ancombc2-E-Modifier-results.qza \
  --output-path exported/ancombc2-E-Modifier-results

!qiime tools export \
  --input-path qiime/ancombc2-C-Modifier-results.qza \
  --output-path exported/ancombc2-C-Modifier-results


In [ ]:

# --- DIRECTORIES TO PROCESS ---
# List all your exported ANCOM-BC2 result directories
EXPORTED_FOLDERS = [
    'exported/ancombc2-Group-results',
    'exported/ancombc2-E-vs-C-results',
    'exported/ancombc2-E-Modifier-results',
    'exported/ancombc2-C-Modifier-results'
]
# ------------------------------
def jsonl_to_tsv(input_filename, output_filename):
    """
    Parses a specific JSONL format (with a schema header) and converts it to TSV.
    """
    try:
        with open(input_filename, 'r', encoding='utf-8') as infile:
            # 1. Read the schema line (first line)
            schema_line = infile.readline()
            if not schema_line:
                print("Error: Input file is empty.")
                return

            schema = json.loads(schema_line)
            
            # Extract the header/field names from the 'fields' array
            # This ensures the correct order for the TSV output
            header = [field['name'] for field in schema.get('fields', [])]

            if not header:
                print("Error: Could not extract header from the schema.")
                return

            # 2. Open the output file for writing TSV
            with open(output_filename, 'w', newline='', encoding='utf-8') as outfile:
                # Use the csv module with the tab character ('\t') as the delimiter
                writer = csv.writer(outfile, delimiter='\t')
                
                # Write the header row
                writer.writerow(header)
                
                # 3. Process the remaining data lines
                for line in infile:
                    if not line.strip(): # Skip empty lines
                        continue
                        
                    try:
                        data_record = json.loads(line)
                        
                        # Extract values in the order defined by the header
                        row_data = [data_record.get(field_name, '') for field_name in header]
                        
                        # Write the data row
                        writer.writerow(row_data)
                        
                    except json.JSONDecodeError as e:
                        print(f"Skipping malformed JSON line: {line.strip()}. Error: {e}", file=sys.stderr)
                        continue
                        
        print(f"Successfully converted {input_filename} to {output_filename}")

    except FileNotFoundError:
        print(f"Error: The file '{input_filename}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
# Run the conversion for all specified directories
for folder in EXPORTED_FOLDERS:
    # Check if the directory exists before trying to convert
    if os.path.isdir(folder):
        print(f"Processing folder: {folder}")
        
        # Use glob to find all files ending with .jsonl in the current folder
        # The ** is for recursive searching (optional, but good practice if needed)
        jsonl_files = glob.glob(os.path.join(folder, '*.jsonl'))

        if not jsonl_files:
            print(f"No .jsonl files found in '{folder}'.")

        for input_file in jsonl_files:
            # 1. Get the base filename (e.g., 'data_01') without the extension
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            
            # 2. Construct the output TSV filename
            # The output file will be placed in the same directory as the input file
            output_file = os.path.join(folder, f"{base_name}.tsv")
            
            # 3. Run the conversion function
            print(f"  Converting '{os.path.basename(input_file)}' to '{os.path.basename(output_file)}'...")
            jsonl_to_tsv(input_file, output_file)
            
    else:
        print(f"Folder not found: '{folder}'. Please check your path.")

print("\n JSONL to .tsv conversion complete. ")
print("\n Combining all .tsv files...")

# Column names you expect in each folder (change if yours are named differently)
EXPECTED_FILES = ["diff.tsv", "lfc.tsv", "p.tsv", "q.tsv", "se.tsv", "W.tsv", "passed_ss.tsv"]

for folder in EXPORTED_FOLDERS:
    if not os.path.isdir(folder):
        print(f" Folder not found: {folder}")
        continue

    print(f"\n Processing folder: {folder}")

    dfs = {}  # dictionary to store all loaded TSVs
    for fname in EXPECTED_FILES:
        file_path = os.path.join(folder, fname)
        if os.path.exists(file_path):
            print(f"   Found: {fname}")
            dfs[fname.replace(".tsv", "")] = pd.read_csv(file_path, sep="\t", index_col=0)
        else:
            print(f"   Missing: {fname}")

    if not dfs:
        print("   No TSV files found — skipping this folder.")
        continue

    # Combine all TSVs into one DataFrame
    combined = pd.concat(dfs, axis=1)

    # Optional: flatten multi-index columns (e.g., diff:ComparisonA)
    combined.columns = [f"{stat}:{col}" for stat, col in combined.columns]

    # Save the final combined file
    output_path = os.path.join(folder, "ancombc2_combined.tsv")
    combined.to_csv(output_path, sep="\t")
    print(f"   Combined table saved: {output_path}")

print("\n All comparison sets processed. ")


In [ ]:
# ----------------- CONFIG -----------------
EXPORTED_FOLDERS = [
    'exported/ancombc2-Group-results',
    'exported/ancombc2-E-vs-C-results',
    'exported/ancombc2-E-Modifier-results',
    'exported/ancombc2-C-Modifier-results'
]

OUTPUT_DIR = "differentials_histograms_barplots"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------- LOAD TAXONOMY -----------------
tax_df = pd.read_csv("exported/exported-taxonomy/taxonomy.tsv", sep="\t", index_col=0)  # index is Feature ID

# ----------------- LOOP THROUGH COMPARISONS -----------------
for folder in EXPORTED_FOLDERS:
    comparison_name = os.path.basename(folder).replace("ancombc2-", "").replace("-results", "")
    print(f"\n📊 Processing {comparison_name}")

    combined_path = os.path.join(folder, "ancombc2_combined.tsv")
    if not os.path.exists(combined_path):
        print(f"❌ Missing {combined_path}, skipping.")
        continue

    df = pd.read_csv(combined_path, sep="\t", index_col=0)

    # --- Map Feature IDs to readable taxa names ---
    df["Taxon"] = df.index.map(lambda x: tax_df.loc[x, "Taxon"] if x in tax_df.index else "Unassigned")

    # --- Find q-value columns ---
    q_cols = [c for c in df.columns if c.startswith("q:")]

    # --- Combined q-value histograms ---
    thresholds = [0.05, 0.1, 0.2]
    if q_cols:
        max_cols = 3
        n_q = len(q_cols)
        n_rows = (n_q + max_cols - 1) // max_cols
        n_cols = min(n_q, max_cols)

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows), squeeze=False)

        for i, q_col in enumerate(q_cols):
            row = i // max_cols
            col = i % max_cols
            ax = axes[row, col]

            for t in thresholds:
                sig_count = (df[q_col] < t).sum()
                print(f"  {q_col}: q < {t:.2f}: {sig_count} taxa")

            sns.histplot(df[q_col].dropna(), bins=40, kde=False, ax=ax)
            ax.axvline(0.05, color="red", linestyle="--", label="0.05")
            ax.axvline(0.1, color="orange", linestyle="--", label="0.10")
            ax.axvline(0.2, color="green", linestyle="--", label="0.20")
            ax.set_title(f"{q_col}")
            ax.set_xlabel("q-value")
            ax.set_ylabel("Count")
            ax.legend()

        # Turn off empty subplots
        total_plots = n_rows * n_cols
        if total_plots > n_q:
            for j in range(n_q, total_plots):
                row = j // max_cols
                col = j % max_cols
                axes[row, col].axis("off")

        plt.suptitle(f"Q-value distributions - {comparison_name}")
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(os.path.join(OUTPUT_DIR, f"{comparison_name}_qval_hist_combined.png"), dpi=300)
        plt.close()
        print(f"✅ Combined Q-value histogram saved: {comparison_name}_qval_hist_combined.png")

    # --- Identify relevant columns for barplot ---
    lfc_cols = [c for c in df.columns if c.startswith("lfc:")]
    se_cols = [c for c in df.columns if c.startswith("se:")]
    import textwrap
    # --- Build barplot data ---
    barplot_data = []
    for lfc_col in lfc_cols:
        comparison = lfc_col.replace("lfc:", "")
        se_col = f"se:{comparison}"
        q_col = f"q:{comparison}"

        lfc = df[lfc_col]
        se = df[se_col] if se_col in df.columns else pd.Series([None]*len(df), index=df.index)
        q = df[q_col] if q_col in df.columns else pd.Series([1.0]*len(df), index=df.index)

        sig = q < 0.1
        for idx in df.index[sig]:
            barplot_data.append({
                "Taxon": df.loc[idx, "Taxon"],  # readable taxon name
                "Comparison": comparison,
                "lfc": lfc.loc[idx],
                "se": se.loc[idx],
                "q": q.loc[idx]
            })

    barplot_df = pd.DataFrame(barplot_data)

    if not barplot_df.empty:
        # Wrap long taxa names at 20 characters
        barplot_df["Taxon_wrapped"] = barplot_df["Taxon"].apply(lambda x: "\n".join(textwrap.wrap(x, 20)))

        # Adjust figure width based on number of unique taxa
        fig_width = max(12, len(barplot_df["Taxon"].unique()) * 0.6)
        plt.figure(figsize=(fig_width, 6))

        # Create the barplot and get axes object
        ax = sns.barplot(
            data=barplot_df,
            x="Taxon_wrapped", y="lfc", hue="Comparison",
            dodge=True, palette="coolwarm", errorbar=None
        )

        # Add error bars correctly for each bar
        for i, row in barplot_df.iterrows():
            if pd.notna(row["se"]):
                # Find the center x-position of the corresponding bar
                bars = [b for b in ax.patches if b.get_height() == row["lfc"] and b.get_x() >= 0]
                if bars:
                    bar = bars[0]  # take first matching bar
                    height = bar.get_height()
                    ax.errorbar(
                        bar.get_x() + bar.get_width() / 2,  # center of bar
                        height,
                        yerr=row["se"],
                        fmt='none', c='black', capsize=3
                    )

        plt.xticks(rotation=80, ha="right")
        plt.ylabel("Log fold change (lfc)")
        plt.title(f"Differential taxa barplot - {comparison_name} (q < 0.1)")
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"{comparison_name}_barplot.png"), dpi=300)
        plt.close()
        print(f"✅ Barplot saved for {comparison_name}")
    else:
        print("⚠️ No significant taxa for barplot (q < 0.1).")



print("\n🎉 Finished! Results in:", OUTPUT_DIR)
